# 2D and 3D Matrix Multiplication
## Setup
### Installation
```pip install ipyparallel```

or 

```pip install -e .[notebook]```

### Start cluster

```ipcluster start -n 4 --engines=MPI --profile mpi```

In [ ]:
import ipyparallel as ipp
rc = ipp.Client(profile='mpi')
rc.wait_for_engines(4)
len(rc)

In [ ]:
%%px
import torch
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
print(f'Hello from rank {rank}!')

In [ ]:
%%px
cart_comm = comm.Create_cart(dims=[2, 2], periods=[True, True], reorder=True)
print(f'Hello from rank {rank}! My coordinates are {cart_comm.Get_coords(rank)}')


In [ ]:
%%px
print(f"Topo: {cart_comm.Get_topo()}")

In [ ]:
%%px
row_comm = cart_comm.Sub([0,1])
row_global_ranks = row_comm.allgather(rank)
print(f'Hello from rank {rank}! In my row, the ranks are {row_global_ranks}')

col_comm = cart_comm.Sub([1,0])
col_global_ranks = col_comm.allgather(rank)
print(f'Hello from rank {rank}! In my col, the ranks are {col_global_ranks}')


In [ ]:
%%px
def as_buffer(x: torch.Tensor):
    return MPI.buffer.fromaddress(x.untyped_storage().data_ptr(), 0)

## Matrix Vector

### y:= Ax

In [ ]:
%%px
A_g = torch.arange(64).reshape(8, 8)
A_l = A_g[col_comm.Get_rank()::2, row_comm.Get_rank()::2].contiguous()
print("A_local:", A_l)

x_g = torch.arange(8).reshape(8, 1)
x_l = x_g[comm.Get_rank()::comm.Get_size(), :].contiguous()
print("x_local:", x_l)


# x_col = torch.zeros(4,1, dtype=torch.long)
# col_comm.Allgather((as_buffer(x_l), 2, MPI.LONG), (as_buffer(x_col), 2, MPI.LONG))
# x_col = x_col.reshape(2,2).T.reshape(4,1).contiguous()
# print("x_gather_col:", x_col)

x_col = torch.zeros(4,1, dtype=torch.long)
data_type = MPI.LONG.Create_vector(2, 1, 2).Create_resized(MPI.LONG.Get_extent()[0], MPI.LONG.Get_extent()[1]).Commit()
col_comm.Allgather((as_buffer(x_l), 2, MPI.LONG), (as_buffer(x_col), 1, data_type))
print("x_gather_col:", x_col)

y_l = A_l @ x_col
print("y_local:", y_l)

y_l = y_l.reshape(2,2).T.reshape(4,1).contiguous()
y_scatter = torch.zeros(2, 1, dtype=torch.long)
row_comm.Reduce_scatter((as_buffer(y_l), 4, MPI.LONG), (as_buffer(y_scatter), 2, MPI.LONG), [2,2], MPI.SUM)
print("y_scatter:", y_scatter)

y_end = torch.zeros(2,2,2, dtype=torch.long)
comm.Allgather((as_buffer(y_scatter), 2, MPI.LONG), (as_buffer(y_end), 2, MPI.LONG))
y_end = y_end.permute(2,1,0).reshape(8,1).contiguous()
print("y_end:", y_end)



In [ ]:
%%px
print("Expected:", A_g @ x_g)
print("Actual:", y_end)
torch.allclose(y_end, A_g @ x_g)

### x = A.T * y

In [ ]:
%%px
A_g = torch.arange(64).reshape(8, 8)
A_l = A_g[col_comm.Get_rank()::2, row_comm.Get_rank()::2].contiguous()
print("A_local:", A_l)

y_g = torch.arange(8).reshape(8, 1)
i = col_comm.Get_rank() + col_comm.Get_size() * row_comm.Get_rank()
y_l = y_g[i::comm.Get_size(), :].contiguous()
print("y_local:", y_l)

y_col = torch.zeros(4,1, dtype=torch.long)
row_comm.Allgather((as_buffer(y_l), 2, MPI.LONG), (as_buffer(y_col), 2, MPI.LONG))
y_col = y_col.reshape(2,2).T.reshape(4,1).contiguous()
print("y_gather_col:", y_col)

x_l = A_l.T @ y_col
print("x_local:", x_l)

x_l = x_l.reshape(2,2).T.reshape(4,1).contiguous()
x_scatter = torch.zeros(2, 1, dtype=torch.long)
col_comm.Reduce_scatter((as_buffer(x_l), 4, MPI.LONG), (as_buffer(x_scatter), 2, MPI.LONG), [2,2], MPI.SUM)
print("x_scatter:", x_scatter)


In [ ]:
%%px
print("expected:", A_g.T @ y_g)

### A := y * x.T + A

In [ ]:
%%px
A_g = torch.arange(64).reshape(8, 8)
A_l = A_g[col_comm.Get_rank()::2, row_comm.Get_rank()::2].contiguous()
print("A_local:", A_l)

x_g = torch.arange(8).reshape(8, 1)
x_l = x_g[comm.Get_rank()::comm.Get_size(), :].contiguous()
print("x_local:", x_l)

y_g = torch.arange(8).reshape(8, 1)
i = col_comm.Get_rank() + col_comm.Get_size() * row_comm.Get_rank()
y_l = y_g[i::comm.Get_size(), :].contiguous()
print("y_local:", y_l)

x_col = torch.zeros(4,1, dtype=torch.long)
col_comm.Allgather((as_buffer(x_l), 2, MPI.LONG), (as_buffer(x_col), 2, MPI.LONG))
x_col = x_col.reshape(2,2).T.reshape(4,1).contiguous()
print("x_gather_col:", x_col)

y_col = torch.zeros(4,1, dtype=torch.long)
row_comm.Allgather((as_buffer(y_l), 2, MPI.LONG), (as_buffer(y_col), 2, MPI.LONG))
y_col = y_col.reshape(2,2).T.reshape(4,1).contiguous()
print("y_gather_col:", y_col)

Z_l = y_col @ x_col.T + A_l
print("Z_local:", Z_l)




In [ ]:
%%px
print(f"Expected: {y_g @ x_g.T + A_g}")

## Create_darray

In [ ]:
%%px
if comm.Get_rank() == 0:
    A = torch.arange(64).reshape(8, 8)
    print(A)
    print(A.dtype)

    darray_type = MPI.LONG.Create_darray(
        4,                    # Size
        1,                      # Rank
        # 2,                      # number of array dimensions (as well as process grid dimensions)
        [64,1],                 # size of the global array
        [MPI.DISTRIBUTE_CYCLIC, MPI.DISTRIBUTE_NONE], # distribution type
        [1, 1], # distribution argument
        [4, 1],                 # size of the process grid
        MPI.ORDER_C,            # array storage order
    ).Commit()

    # comm.Send(buf=[as_buffer(A), 8, MPI.LONG], dest=1)
    comm.Send([as_buffer(A), 1, darray_type], dest=1, tag=55)

    darray_type.Free() 

elif comm.Get_rank() == 1:
    A = torch.zeros(4, 4, dtype=torch.int64)

    # comm.Recv(buf=[as_buffer(A), 8, MPI.LONG], source=0)
    comm.Recv([as_buffer(A), 16, MPI.LONG], source=0, tag=55)
    print(A)



## Large Item Count

In [ ]:
%%px
import tensorcraft as tc
comm = MPI.COMM_WORLD
## Let's find the maximum element count on my mpi implementation
options = [torch.iinfo(torch.int32).max]

for possible_max in options:
    print(f"Trying {possible_max}")
    if comm.Get_rank() == 0:
        A = torch.ones(possible_max, dtype=torch.bool)
        print(A.dtype)
        print(A[:10])

        print(f"Sending {possible_max} elements, {possible_max / 10**9} Gb")

    else:
        A = torch.zeros(possible_max, dtype=torch.bool)

    comm.Bcast(buf=[tc.mpi4torch.as_buffer(A), possible_max, MPI.BOOL], root=0)

    if comm.Get_rank() == 0:
        print("Sent!")
    else:
        print("Received!")
        print(A[:10])

# Interweave allgather

In [ ]:
import ipyparallel as ipp
rc = ipp.Client(profile='mpi')
rc.wait_for_engines(4)
len(rc)

4

In [ ]:
%%px
import torch
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
print(f'Hello from rank {rank}!')

%px:   0%|          | 0/4 [00:03<?, ?tasks/s]

[stdout:3] Hello from rank 3!


%px:  25%|██▌       | 1/4 [00:03<00:00,  9.53tasks/s]

[stdout:1] Hello from rank 1!


%px:  50%|█████     | 2/4 [00:03<00:00,  6.11tasks/s]

[stdout:0] Hello from rank 0!


[stdout:2] Hello from rank 2!


%px: 100%|██████████| 4/4 [00:03<00:00,  1.08tasks/s]


In [ ]:
%%px
import logging
import tensorcraft as tc

tc.set_logger_config(log_rank=True, level=logging.DEBUG)

x = torch.arange(40).reshape(2, 10, 2)
mesh = torch.Size([2,2])
dist = tc.mpi.MPIMultiAxisDist(mesh, (None, (0,1), None), 1)

x_local = dist.apply(x, rank)
print(x_local)
print(x_local.shape)
print(x_local.dtype)
print(x_local.is_contiguous())

%px:   0%|          | 0/4 [00:01<?, ?tasks/s]

[stdout:1] [2025-05-02 14:18:33,045][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/4:Processor multi index: torch.Size([0, 1])
[2025-05-02 14:18:33,047][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/4:Missing elements: [0, 1, 0]
[2025-05-02 14:18:33,048][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/4:N blocks per axis: [1, 10, 1]
[2025-05-02 14:18:33,050][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/4:Padded tensor shape: torch.Size([2, 11, 2])
[2025-05-02 14:18:33,054][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/4:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-02 14:18:33,056][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/4:Reshape tuple: [1, 2, 11, 1, 1, 2]
[2025-05-02 14:18:33,062][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/4:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(1), None, 4), slice(None, None, None), sl

[stdout:3] [2025-05-02 14:18:33,106][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/4:Processor multi index: torch.Size([1, 1])
[2025-05-02 14:18:33,108][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/4:Missing elements: [0, 1, 0]
[2025-05-02 14:18:33,111][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/4:N blocks per axis: [1, 10, 1]
[2025-05-02 14:18:33,116][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/4:Padded tensor shape: torch.Size([2, 11, 2])
[2025-05-02 14:18:33,122][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/4:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-02 14:18:33,125][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/4:Reshape tuple: [1, 2, 11, 1, 1, 2]
[2025-05-02 14:18:33,129][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/4:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(3), None, 4), slice(None, None, None), sl

%px:  25%|██▌       | 1/4 [00:01<00:00,  9.61tasks/s]

[stdout:0] [2025-05-02 14:18:33,146][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/4:Processor multi index: torch.Size([0, 0])
[2025-05-02 14:18:33,148][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/4:Missing elements: [0, 1, 0]
[2025-05-02 14:18:33,150][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/4:N blocks per axis: [1, 10, 1]
[2025-05-02 14:18:33,152][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/4:Padded tensor shape: torch.Size([2, 11, 2])
[2025-05-02 14:18:33,156][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/4:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-02 14:18:33,158][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/4:Reshape tuple: [1, 2, 11, 1, 1, 2]
[2025-05-02 14:18:33,162][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/4:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(0), None, 4), slice(None, None, None), sl

%px:  75%|███████▌  | 3/4 [00:01<00:00, 15.24tasks/s]

[stdout:2] [2025-05-02 14:18:33,254][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/4:Processor multi index: torch.Size([1, 0])
[2025-05-02 14:18:33,259][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/4:Missing elements: [0, 1, 0]
[2025-05-02 14:18:33,262][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/4:N blocks per axis: [1, 10, 1]
[2025-05-02 14:18:33,263][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/4:Padded tensor shape: torch.Size([2, 11, 2])
[2025-05-02 14:18:33,268][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/4:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-02 14:18:33,270][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/4:Reshape tuple: [1, 2, 11, 1, 1, 2]
[2025-05-02 14:18:33,279][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/4:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(2), None, 4), slice(None, None, None), sl

%px: 100%|██████████| 4/4 [00:01<00:00,  2.48tasks/s]


In [ ]:
%%px
new_dist, post_gather = dist.apply_allgather(x.shape, x_local, comm, gather_mesh_dim=0)
print(post_gather)

[stdout:3] [2025-05-02 14:18:33,506][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Tensor axis: 1
[2025-05-02 14:18:33,508][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mesh axis: 0
[2025-05-02 14:18:33,509][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mappings: (0, 1)
[2025-05-02 14:18:33,512][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Tensor axis: 1
[2025-05-02 14:18:33,514][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mesh axis: 0
[2025-05-02 14:18:33,515][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mappings: (0, 1)
[2025-05-02 14:18:33,522][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R3/4:New distribution: D_[2,2]⊥{∅,1,∅}(∅,1,∅), new shape: torch.Size([2, 5, 2])
[2025-05-02 14:18:33,525][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R3/4:Local tensor shape: torch.Size([2, 2, 2])
[2025-05-02 14

[stdout:0] [2025-05-02 14:18:33,505][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Tensor axis: 1
[2025-05-02 14:18:33,509][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mesh axis: 0
[2025-05-02 14:18:33,510][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mappings: (0, 1)
[2025-05-02 14:18:33,515][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Tensor axis: 1
[2025-05-02 14:18:33,518][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mesh axis: 0
[2025-05-02 14:18:33,522][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mappings: (0, 1)
[2025-05-02 14:18:33,528][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R0/4:New distribution: D_[2,2]⊥{∅,1,∅}(∅,1,∅), new shape: torch.Size([2, 5, 2])
[2025-05-02 14:18:33,535][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R0/4:Local tensor shape: torch.Size([2, 3, 2])
[2025-05-02 14

[stdout:1] [2025-05-02 14:18:33,512][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Tensor axis: 1
[2025-05-02 14:18:33,516][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mesh axis: 0
[2025-05-02 14:18:33,522][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mappings: (0, 1)
[2025-05-02 14:18:33,536][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Tensor axis: 1
[2025-05-02 14:18:33,544][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mesh axis: 0
[2025-05-02 14:18:33,553][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mappings: (0, 1)
[2025-05-02 14:18:33,565][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R1/4:New distribution: D_[2,2]⊥{∅,1,∅}(∅,1,∅), new shape: torch.Size([2, 5, 2])
[2025-05-02 14:18:33,579][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R1/4:Local tensor shape: torch.Size([2, 3, 2])
[2025-05-02 14

[stdout:2] [2025-05-02 14:18:33,526][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Tensor axis: 1
[2025-05-02 14:18:33,530][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mesh axis: 0
[2025-05-02 14:18:33,531][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mappings: (0, 1)
[2025-05-02 14:18:33,536][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Tensor axis: 1
[2025-05-02 14:18:33,544][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mesh axis: 0
[2025-05-02 14:18:33,548][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mappings: (0, 1)
[2025-05-02 14:18:33,553][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R2/4:New distribution: D_[2,2]⊥{∅,1,∅}(∅,1,∅), new shape: torch.Size([2, 5, 2])
[2025-05-02 14:18:33,558][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R2/4:Local tensor shape: torch.Size([2, 2, 2])
[2025-05-02 14

[stderr:3] /home/juanpedroghm/code/tensorcraft/tensorcraft/util/axis_utils.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index_tensor = torch.tensor(index)


[stderr:1] /home/juanpedroghm/code/tensorcraft/tensorcraft/util/axis_utils.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index_tensor = torch.tensor(index)


[stderr:0] /home/juanpedroghm/code/tensorcraft/tensorcraft/util/axis_utils.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index_tensor = torch.tensor(index)


[stderr:2] /home/juanpedroghm/code/tensorcraft/tensorcraft/util/axis_utils.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index_tensor = torch.tensor(index)


In [ ]:
%%px
post_gather

Out[2:4]: 
tensor([[[ 0,  1],
         [ 4,  5],
         [ 8,  9],
         [12, 13],
         [16, 17]],

        [[20, 21],
         [24, 25],
         [28, 29],
         [32, 33],
         [36, 37]]])

Out[0:4]: 
tensor([[[ 0,  1],
         [ 4,  5],
         [ 8,  9],
         [12, 13],
         [16, 17]],

        [[20, 21],
         [24, 25],
         [28, 29],
         [32, 33],
         [36, 37]]])

Out[1:4]: 
tensor([[[ 2,  3],
         [ 6,  7],
         [10, 11],
         [14, 15],
         [18, 19]],

        [[22, 23],
         [26, 27],
         [30, 31],
         [34, 35],
         [38, 39]]])

Out[3:4]: 
tensor([[[ 2,  3],
         [ 6,  7],
         [10, 11],
         [14, 15],
         [18, 19]],

        [[22, 23],
         [26, 27],
         [30, 31],
         [34, 35],
         [38, 39]]])

In [ ]:
%%px
new_dist, post_gather = dist.apply_allgather(x.shape, x_local, comm, gather_mesh_dim=1)
print(post_gather)

[stdout:0] [2025-05-02 14:19:16,455][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Tensor axis: 1
[2025-05-02 14:19:16,457][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mesh axis: 1
[2025-05-02 14:19:16,458][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mappings: (0, 1)
[2025-05-02 14:19:16,461][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Tensor axis: 1
[2025-05-02 14:19:16,464][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mesh axis: 1
[2025-05-02 14:19:16,465][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R0/4:Mappings: (0, 1)
[2025-05-02 14:19:16,466][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R0/4:New distribution: D_[2,2]⊥{∅,0,∅}(∅,2,∅), new shape: torch.Size([2, 6, 2])
[2025-05-02 14:19:16,467][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R0/4:Local tensor shape: torch.Size([2, 3, 2])
[2025-05-02 14

[stdout:3] [2025-05-02 14:19:16,456][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Tensor axis: 1
[2025-05-02 14:19:16,458][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mesh axis: 1
[2025-05-02 14:19:16,459][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mappings: (0, 1)
[2025-05-02 14:19:16,461][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Tensor axis: 1
[2025-05-02 14:19:16,463][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mesh axis: 1
[2025-05-02 14:19:16,464][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R3/4:Mappings: (0, 1)
[2025-05-02 14:19:16,465][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R3/4:New distribution: D_[2,2]⊥{∅,0,∅}(∅,2,∅), new shape: torch.Size([2, 4, 2])
[2025-05-02 14:19:16,467][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R3/4:Local tensor shape: torch.Size([2, 2, 2])
[2025-05-02 14

[stdout:1] [2025-05-02 14:19:16,457][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Tensor axis: 1
[2025-05-02 14:19:16,460][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mesh axis: 1
[2025-05-02 14:19:16,461][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mappings: (0, 1)
[2025-05-02 14:19:16,465][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Tensor axis: 1
[2025-05-02 14:19:16,465][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mesh axis: 1
[2025-05-02 14:19:16,465][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R1/4:Mappings: (0, 1)
[2025-05-02 14:19:16,467][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R1/4:New distribution: D_[2,2]⊥{∅,0,∅}(∅,2,∅), new shape: torch.Size([2, 6, 2])
[2025-05-02 14:19:16,468][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R1/4:Local tensor shape: torch.Size([2, 3, 2])
[2025-05-02 14

[stdout:2] [2025-05-02 14:19:16,465][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Tensor axis: 1
[2025-05-02 14:19:16,471][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mesh axis: 1
[2025-05-02 14:19:16,472][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mappings: (0, 1)
[2025-05-02 14:19:16,473][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Tensor axis: 1
[2025-05-02 14:19:16,473][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mesh axis: 1
[2025-05-02 14:19:16,474][tensorcraft.distributions.multi_axis][allgather][DEBUG] - R2/4:Mappings: (0, 1)
[2025-05-02 14:19:16,475][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R2/4:New distribution: D_[2,2]⊥{∅,0,∅}(∅,2,∅), new shape: torch.Size([2, 4, 2])
[2025-05-02 14:19:16,478][tensorcraft.mpi.distributions.multi_axis_dist][_apply_single_axis_allgather][DEBUG] - R2/4:Local tensor shape: torch.Size([2, 2, 2])
[2025-05-02 14

In [ ]:
%%px
print(post_gather)

[stdout:2] tensor([[[ 4,  5],
         [ 6,  7],
         [12, 13],
         [14, 15]],

        [[24, 25],
         [26, 27],
         [32, 33],
         [34, 35]]])


[stdout:0] tensor([[[ 0,  1],
         [ 2,  3],
         [ 8,  9],
         [10, 11],
         [16, 17],
         [18, 19]],

        [[20, 21],
         [22, 23],
         [28, 29],
         [30, 31],
         [36, 37],
         [38, 39]]])


[stdout:3] tensor([[[ 4,  5],
         [ 6,  7],
         [12, 13],
         [14, 15]],

        [[24, 25],
         [26, 27],
         [32, 33],
         [34, 35]]])


[stdout:1] tensor([[[ 0,  1],
         [ 2,  3],
         [ 8,  9],
         [10, 11],
         [16, 17],
         [18, 19]],

        [[20, 21],
         [22, 23],
         [28, 29],
         [30, 31],
         [36, 37],
         [38, 39]]])


# Permute

In [1]:
import ipyparallel as ipp
rc = ipp.Client(profile='mpi')
rc.wait_for_engines(8)
len(rc)

8

In [2]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
w_size = comm.Get_size()

import torch
import logging
import tensorcraft as tc

tc.set_logger_config(log_rank=True, level=logging.DEBUG)

shape = torch.Size([2, 37, 2])
x = torch.arange(shape.numel()).reshape(shape)

mesh = torch.Size([2,2,2])
dist = tc.mpi.MPIMultiAxisDist(mesh, (None, (0,2,1), None),1)

x_local = dist.apply(x, rank)
print(x_local)
print(x_local.shape)
print(x_local.dtype)
print(x_local.is_contiguous())

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:2] [2025-05-30 20:57:14,505][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Processor multi index: torch.Size([0, 1, 0])
[2025-05-30 20:57:14,506][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,506][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,507][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,509][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,510][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,511][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(1), None, 8), slice(None, None, None),

[stdout:5] [2025-05-30 20:57:14,684][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R5/8:Processor multi index: torch.Size([1, 0, 1])
[2025-05-30 20:57:14,685][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R5/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,686][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R5/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,687][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R5/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,689][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R5/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,691][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R5/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,693][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R5/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(6), None, 8), slice(None, None, None),

[stdout:7] [2025-05-30 20:57:14,691][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R7/8:Processor multi index: torch.Size([1, 1, 1])
[2025-05-30 20:57:14,693][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R7/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,694][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R7/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,695][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R7/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,698][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R7/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,699][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R7/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,703][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R7/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(7), None, 8), slice(None, None, None),

[stdout:6] [2025-05-30 20:57:14,769][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R6/8:Processor multi index: torch.Size([1, 1, 0])
[2025-05-30 20:57:14,770][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R6/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,771][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R6/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,771][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R6/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,773][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R6/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,773][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R6/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,775][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R6/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(5), None, 8), slice(None, None, None),

[stdout:4] [2025-05-30 20:57:14,800][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Processor multi index: torch.Size([1, 0, 0])
[2025-05-30 20:57:14,800][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,801][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,802][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,803][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,805][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,808][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(4), None, 8), slice(None, None, None),

[stdout:1] [2025-05-30 20:57:14,808][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Processor multi index: torch.Size([0, 0, 1])
[2025-05-30 20:57:14,808][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,809][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,810][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,811][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,813][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,815][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(2), None, 8), slice(None, None, None),

[stdout:3] [2025-05-30 20:57:14,880][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Processor multi index: torch.Size([0, 1, 1])
[2025-05-30 20:57:14,881][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,881][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,882][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,883][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,884][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,886][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(3), None, 8), slice(None, None, None),

[stdout:0] [2025-05-30 20:57:14,913][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Processor multi index: torch.Size([0, 0, 0])
[2025-05-30 20:57:14,918][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Missing elements: [0, 1, 0]
[2025-05-30 20:57:14,923][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:N blocks per axis: [1, 37, 1]
[2025-05-30 20:57:14,924][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Padded tensor shape: torch.Size([2, 38, 2])
[2025-05-30 20:57:14,926][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Permute tuple: (0, 2, 4, 1, 3, 5)
[2025-05-30 20:57:14,926][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Reshape tuple: [1, 2, 38, 1, 1, 2]
[2025-05-30 20:57:14,928][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Tile Slices: [slice(None, None, None), slice(None, None, None), slice(tensor(0), None, 8), slice(None, None, None),

In [4]:
%%px
dist.localShape(x.shape, rank)

Out[0:3]: torch.Size([2, 5, 2])

Out[6:3]: torch.Size([2, 4, 2])

Out[3:3]: torch.Size([2, 5, 2])

Out[1:3]: torch.Size([2, 5, 2])

Out[4:3]: torch.Size([2, 5, 2])

Out[7:3]: torch.Size([2, 4, 2])

Out[5:3]: torch.Size([2, 4, 2])

Out[2:3]: torch.Size([2, 5, 2])

In [3]:
%%px
expected_dist, _, _ = dist.permute(x.shape, mesh_dims=[0,1])
expected_dist.localShape(x.shape, rank)

Out[2:2]: torch.Size([2, 5, 2])

Out[1:2]: torch.Size([2, 5, 2])

Out[0:2]: torch.Size([2, 5, 2])

Out[3:2]: torch.Size([2, 4, 2])

Out[5:2]: torch.Size([2, 5, 2])

Out[4:2]: torch.Size([2, 5, 2])

Out[7:2]: torch.Size([2, 4, 2])

Out[6:2]: torch.Size([2, 4, 2])

In [5]:
%%px
new_dist, new_local = dist.apply_permute(x.shape, x_local, comm, mesh_dims=[0,1])
new_local

[stdout:2] [2025-05-30 21:03:12,130][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R2/8:tensor([[[  2,   3],
         [ 18,  19],
         [ 34,  35],
         [ 50,  51],
         [ 66,  67]],

        [[ 76,  77],
         [ 92,  93],
         [108, 109],
         [124, 125],
         [140, 141]]])


[stdout:1] [2025-05-30 21:03:12,130][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R1/8:tensor([[[  4,   5],
         [ 20,  21],
         [ 36,  37],
         [ 52,  53],
         [ 68,  69]],

        [[ 78,  79],
         [ 94,  95],
         [110, 111],
         [126, 127],
         [142, 143]]])
[2025-05-30 21:03:12,134][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R1/8:1/8 - Creating cart
[2025-05-30 21:03:12,157][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R1/8:1/1/8 - What's the target?
[2025-05-30 21:03:12,158][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R1/8:1/1/8 - Current sub_midx [0, 0, 1]
[2025-05-30 21:03:12,158][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R1/8:1/1/8 - Target sub_midx tensor([1, 0, 0])
[2025-05-30 21:03:12,160][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R1/8:1/1/8 - Target sub_rank 4
[2025-05-30 2

[stdout:6] [2025-05-30 21:03:12,131][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R6/8:tensor([[[ 10,  11],
         [ 26,  27],
         [ 42,  43],
         [ 58,  59]],

        [[ 84,  85],
         [100, 101],
         [116, 117],
         [132, 133]]])
[2025-05-30 21:03:12,132][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R6/8:6/8 - Creating cart
[2025-05-30 21:03:12,157][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R6/8:6/6/8 - What's the target?
[2025-05-30 21:03:12,158][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R6/8:6/6/8 - Current sub_midx [1, 1, 0]
[2025-05-30 21:03:12,158][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R6/8:6/6/8 - Target sub_midx tensor([0, 1, 1])
[2025-05-30 21:03:12,159][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R6/8:6/6/8 - Target sub_rank 3
[2025-05-30 21:03:12,169][tensorcraft.mpi.mpi_utils][te

[stdout:7] [2025-05-30 21:03:12,133][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R7/8:tensor([[[ 14,  15],
         [ 30,  31],
         [ 46,  47],
         [ 62,  63]],

        [[ 88,  89],
         [104, 105],
         [120, 121],
         [136, 137]]])
[2025-05-30 21:03:12,135][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R7/8:7/8 - Creating cart
[2025-05-30 21:03:12,157][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R7/8:7/7/8 - What's the target?
[2025-05-30 21:03:12,158][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R7/8:7/7/8 - Current sub_midx [1, 1, 1]
[2025-05-30 21:03:12,159][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R7/8:7/7/8 - Target sub_midx tensor([1, 1, 1])
[2025-05-30 21:03:12,160][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R7/8:7/7/8 - Target sub_rank 7
[2025-05-30 21:03:12,169][tensorcraft.mpi.distributions

[stdout:5] [2025-05-30 21:03:12,134][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R5/8:tensor([[[ 12,  13],
         [ 28,  29],
         [ 44,  45],
         [ 60,  61]],

        [[ 86,  87],
         [102, 103],
         [118, 119],
         [134, 135]]])
[2025-05-30 21:03:12,138][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R5/8:5/8 - Creating cart
[2025-05-30 21:03:12,161][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R5/8:5/5/8 - What's the target?
[2025-05-30 21:03:12,163][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R5/8:5/5/8 - Current sub_midx [1, 0, 1]
[2025-05-30 21:03:12,164][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R5/8:5/5/8 - Target sub_midx tensor([1, 0, 1])
[2025-05-30 21:03:12,165][tensorcraft.mpi.distributions.multi_axis_dist][apply_permute][DEBUG] - R5/8:5/5/8 - Target sub_rank 5
[2025-05-30 21:03:12,168][tensorcraft.mpi.distributions

...

[6:execute]
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Cell In[4], line 1
----> 1 new_dist, new_local = dist.apply_permute(x.shape, x_local, comm, mesh_dims=[0,1])
      2 new_local

File ~/code/tensorcraft/tensorcraft/mpi/distributions/multi_axis_dist.py:569, in MPIMultiAxisDist.apply_permute(self, global_shape, local_tensor, comm, mesh_dims)
    566 target_tensor = torch.empty(expected_shape, dtype=local_tensor.dtype)
    567 target_buffer = tensor2mpiBuffer(target_tensor)
--> 569 sub_comm.Sendrecv(buffer_tuple, dest=target_sub_rank, recvbuf=target_tensor)
    570 log.debug(target_tensor)
    572 local_tensor = target_tensor

File src/mpi4py/MPI.src/Comm.pyx:470, in mpi4py.MPI.Comm.Sendrecv()

Exception: MPI_ERR_TRUNCATE: message truncated


%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

Received Keyboard Interrupt. Sending signal SIGINT to engines...


: 

: 

In [ ]:
%%px
expected_local = new_dist.apply(x, rank)
# torch.all(expected_local ==  new_local)
expected_local

[stdout:2] [2025-05-30 20:33:50,980][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Processor multi index: torch.Size([0, 1, 0])
[2025-05-30 20:33:50,980][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Processor multi index: torch.Size([0, 1, 0])
[2025-05-30 20:33:50,980][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Processor multi index: torch.Size([0, 1, 0])
[2025-05-30 20:33:50,990][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,990][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,990][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,995][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:N blocks per axis: [2, 10]
[2025-05-30 20:33:50,995][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R2/8:

[stdout:0] [2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Processor multi index: torch.Size([0, 0, 0])
[2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Processor multi index: torch.Size([0, 0, 0])
[2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Processor multi index: torch.Size([0, 0, 0])
[2025-05-30 20:33:50,985][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,985][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,985][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,990][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:N blocks per axis: [2, 10]
[2025-05-30 20:33:50,990][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R0/8:

[stdout:4] [2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Processor multi index: torch.Size([1, 0, 0])
[2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Processor multi index: torch.Size([1, 0, 0])
[2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Processor multi index: torch.Size([1, 0, 0])
[2025-05-30 20:33:50,989][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,989][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,989][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,993][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:N blocks per axis: [2, 10]
[2025-05-30 20:33:50,993][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R4/8:

[stdout:3] [2025-05-30 20:33:50,981][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Processor multi index: torch.Size([0, 1, 1])
[2025-05-30 20:33:50,981][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Processor multi index: torch.Size([0, 1, 1])
[2025-05-30 20:33:50,981][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Processor multi index: torch.Size([0, 1, 1])
[2025-05-30 20:33:50,992][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,992][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,992][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,999][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:N blocks per axis: [2, 10]
[2025-05-30 20:33:50,999][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R3/8:

[stdout:1] [2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Processor multi index: torch.Size([0, 0, 1])
[2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Processor multi index: torch.Size([0, 0, 1])
[2025-05-30 20:33:50,982][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Processor multi index: torch.Size([0, 0, 1])
[2025-05-30 20:33:50,987][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,987][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,987][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:Missing elements: [1, 2]
[2025-05-30 20:33:50,991][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:N blocks per axis: [2, 10]
[2025-05-30 20:33:50,991][tensorcraft.mpi.distributions.multi_axis_dist][apply][DEBUG] - R1/8:

...

Out[0:13]: tensor(True)

Out[1:13]: tensor(True)

Out[4:13]: tensor(True)

Out[7:13]: tensor(True)

Out[3:13]: tensor(True)